In [1]:
!pwd

/content


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
os.chdir("drive/My Drive/textcnn_updated/textcnn")

In [4]:
!ls

clean_data_abstract_ln_200.pkl	      linked_diseases_abstract.csv
clean_data_disease_first_2.pkl	      NCBI_corpus
clean_data_disease_first_last.pkl     NCBI_corpus.zip
clean_data_disease_last_2.pkl	      NCBItrainset.csv
clean_data_superclass_3_4.pkl	      network.py
clean_data_superclass_4_5.pkl	      predictions-Copy1.ipynb
clean_data_superclass_first_2.pkl     predictions.ipynb
clean_data_superclass_first_last.pkl  __pycache__
clean_data_superclass_last_2.pkl      random.csv
clean_data.txt			      random.gsheet
components.py			      results_bilstm_superclass.csv
dropout_layers.py		      results_cnn_disease_names.csv
first_review.png		      results_cnn_superclass.csv
glove.42B.300d.txt		      text.txt
glove.42B.300d.txt.1		      Untitled.ipynb
glove.42B.300d.zip		      utils
glove.42B.300d.zip.1


In [5]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5599172204387551880, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4743165672180126383
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10065612793185562150
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11276946637
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15822882691043262764
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer
import csv,pandas,re
from keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding
from keras.layers import Embedding
import numpy as np
from keras.models import Model

import time, os
#os.environ["MKL_THREADING_LAYER"] = "GNU"
import components as com

# Torch imports
import torch
import torch.nn as nn
from torch import optim
import network as net
import traceback
from utils import tensor_utils as tu

from tqdm import tqdm


import json
import requests
from pprint import pprint

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences



device = torch.device("cuda")
cpu = torch.device("cpu")

Using TensorFlow backend.


In [0]:
from __future__ import unicode_literals, print_function
import spacy

nlp = spacy.load('en')
# Use this command to download the english language model if error comes. ! python -m spacy download en

In [0]:
def get_sentences_from_abstract(abstract):
    doc = nlp(abstract)
    sentences = [sent.string.strip() for sent in doc.sents]
    
    return sentences

def keep_sentences(sentences, index, start=2, end=4, sentence_type='last'):
#     new_abstract = []
    if sentence_type == 'first':
        new_abstract = sentences[:index]
    elif sentence_type == 'last':
        new_abstract = sentences[-index:]
    elif sentence_type == 'middle':
        new_abstract = sentences[start:end]
    elif sentence_type == 'first_last':
        new_abstract = sentences[:start] + sentences[-end:]
    
    return new_abstract

def join_lists(new_abstract):
    new_abstract = ''.join(map(str, new_abstract))
#     new_abstract.join('.')
    return new_abstract

In [0]:
debug = True
MAX_SEQUENCE_LENGTH = 200


class BadResponseException(Exception):
    pass


class Timer:
    """ Simple block which can be called as a context, to know the time of a block. """
    def __enter__(self):
        self.start = time.perf_counter()
        return self

    def __exit__(self, *args):
        self.end = time.perf_counter()
        self.interval = self.end - self.start

In [0]:
#data helpers - copied from https://github.com/bhaveshoswal/CNN-text-classification-keras
def clean_str(string):
    """
    Tokenization/string cleaning for datasets.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ", string)
    string = re.sub(r"\)", " ", string)
    string = re.sub(r"\?", " ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

def _get_class_for_doi_(doi:str) -> list:
    url = "http://id.nlm.nih.gov/mesh/sparql"
    query = """
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
                PREFIX owl: <http://www.w3.org/2002/07/owl#>
                PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
                PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
                PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
                PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
                PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
                PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
                PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
                SELECT DISTINCT ?p ?label ?uri 
                WHERE { mesh:%s meshv:broaderDescriptor* ?uri .
                    ?uri rdfs:label ?p.
                    FILTER NOT EXISTS{
                    ?uri meshv:broaderDescriptor ?x
                    }
                }
                """ % doi
    querystring = {"query":query, 'format':'json'}
    payload = ""
    headers = {
        'cache-control': "no-cache",
        'Postman-Token': "c9ace615-1b99-4caf-bcad-97ce1060973e"
        }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    return [x['uri']['value'] for x in response.json()['results']['bindings']]


def get_classes_for_dois(dois:list) -> list:
    cls = []
    for doi in dois:
        cls += _get_class_for_doi_(doi)
    
    return list(set(cls))

In [0]:
def get_terms_of_classes(list_terms) -> list:
    url = "http://id.nlm.nih.gov/mesh/sparql"
    query = """
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
            PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
            PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
            PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
            PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
            PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
            PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
            SELECT DISTINCT ?class
            WHERE { mesh:%s rdfs:label ?class . }
            """%list_terms
    querystring = {"query":query, 'format':'json'}
    payload = ""
    headers = {
        'cache-control': "no-cache",
        'Postman-Token': "c9ace615-1b99-4caf-bcad-97ce1060973e"
        }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    return [x['class']['value'] for x in response.json()['results']['bindings']]
        

In [0]:
def get_list_for_terms(dois:list) -> list:
    cls = []
    for doi in dois:
        cls += get_terms_of_classes(doi.replace("http://id.nlm.nih.gov/mesh/", ""))
    
    return list(set(cls))

In [0]:
df = pandas.read_csv('linked_diseases_abstract.csv')
data = [x for x in df.to_records(index=False)] 
def read_till_next(data,index):
    temp = []
    temp_loc = 0
    for d in data[index+1:]:
        if 'a' in d[1] or 't' in d[1]:
            return temp,temp_loc
        else:
            temp_loc = temp_loc + 1
            temp.append([d[2],d[3],d[4]])
#             print([d[2],d[3],d[4]])
#             break
    
    return temp,temp_loc

loc = 0
final_data = []
for index,d in enumerate(data):
    if '|t|' in d[1]:
        continue
    elif loc != 0:
        loc = loc - 1
        continue
    else:
        if '|a|' in d[1]:
            temp_holder = {
                'abstract' : d[1],
                'keys' : []
            }
            keys,loc = read_till_next(data,index)
            temp_holder['keys'] = keys
            final_data.append(temp_holder)
clean_data = []
for f in final_data:
    '''
       a =  get_sentence(f['abstract'])
       a = remove_sentencs(a) #a[:2] + a[-3:]
       a = a.join(".")
    
    '''
    
    sent = get_sentences_from_abstract(abstract=f['abstract'].split('|')[2])    # get sentences
    sent_keep = keep_sentences(sentences=sent, index=3,sentence_type='first' )  # sentences to keep
    abst = join_lists(new_abstract=sent_keep)   # as we get the list of sentences to keep, joining them as one string
#     abstract = clean_str(f['abstract'].split('|')[2])
    abstract = clean_str(abst)
    disease = []
    for k in f['keys']:
        disease.append(k[-1])
    clean_data.append((abstract,list(set(disease))))

In [0]:
#Now changing the hiearchy of the diseases dataset where we only consider Diseases ontology 
clean_data_upper_class = []
us = []
for c in tqdm(clean_data):
    try:
        labels = []
        for l in c[1]:
            if 'D' == l[0]:
                labels = labels + l.split('|')
        #change the class hiearchy 
        new_labels = list(set(get_classes_for_dois(labels)))
        clean_data_upper_class.append((c[0],new_labels))
    except:
        us.append(c)

clean_data = clean_data_upper_class

100%|██████████| 793/793 [07:32<00:00,  1.42it/s]


In [0]:
targets = [tar[1] for tar in clean_data]

In [0]:
import pickle
with open ('clean_data_superclass_first_last.pkl', 'rb') as fp:
  clean_data = pickle.load(fp)

In [16]:
clean_data[0]

('wnt signalling orchestrates a number of developmental programs this percentage of ctnnb1 mutations is greater than in all other human tumours examined thus far , and directly implicates beta catenin lef misregulation as the major cause of hair matrix cell tumorigenesis in humans',
 ['http://id.nlm.nih.gov/mesh/D004066',
  'http://id.nlm.nih.gov/mesh/D009369',
  'http://id.nlm.nih.gov/mesh/D017437',
  'http://id.nlm.nih.gov/mesh/D009358'])

In [0]:
#creating id map of class that is each label is given a unique id
all_class = list(set([g for f in clean_data for g in f[1]]))
all_class_to_id = {}
for index,value in enumerate(all_class):
    all_class_to_id[value] = index

texts = [f[0] for f in clean_data]

#Tokenize,idfy,pad
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [0]:
seq_len = [len(text.split()) for text in texts]

In [0]:
print("max length: ", np.max(seq_len))
print("mean: ",np.mean(seq_len))
print("min length: ", np.min(seq_len))
print("median: ", np.median(seq_len))

max length:  118
mean:  46.10013717421125
min length:  3
median:  45.0


In [0]:
ids = []
for i in all_class:
    ids.append(i.replace("http://id.nlm.nih.gov/mesh/", ""))
list_ids=pandas.DataFrame(ids)

In [0]:
targets = get_list_for_terms(all_class)

In [0]:
with open(f'clean_data_abstract_ln_first_three_{MAX_SEQUENCE_LENGTH}.pkl', 'wb') as fp:
  pickle.dump(clean_data, fp)

In [18]:
#constructing the label matrix 
label_matrix = np.zeros((len(all_class),len(all_class)))
for index,value in enumerate(all_class):
    label_matrix[index][index] = 1
    
#Testing 
label_matrix[all_class_to_id[all_class[1]]]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
def create_multi_hot_label(label_matrix,labels):
    '''
    
        label_matrix --> matrix having one hot vectors
        labels --> [idfy]--> [2,5,6]
        return [0,0,1,0,0,1....]
    '''
    vector = np.zeros((label_matrix.shape[0]))
    for i in labels:
        vector = vector + label_matrix[all_class_to_id[all_class[i]]]
    
    return vector

In [0]:
label_vector = []
for f in clean_data:
    label_id = [all_class_to_id[l] for l in f[1]]
    multi_hot_vector = create_multi_hot_label(label_matrix,label_id)
    label_vector.append(multi_hot_vector)

In [0]:
X = data
Y = label_vector
split_index = int(len(X)*.80)
train_x,val_x = X[:split_index],X[split_index:]
train_y,val_y = Y[:split_index],Y[split_index:]

In [0]:
class CNNDot(Model):

    def __init__(self, _parameter_dict, _word_to_id, _device, _pointwise=False, _debug=False):

        self.debug = _debug
        self.parameter_dict = _parameter_dict
        self.device = _device
        self.pointwise = _pointwise
        self.word_to_id = _word_to_id
        self.number_of_labels = self.parameter_dict['number_of_labels']

        if self.debug:
            print("Init Models")

        self.encoder = com.CNN(_vectors=self.parameter_dict['vectors'], _vocab_size=self.parameter_dict['vocab_size'] ,
                           _embedding_dim = self.parameter_dict['embedding_dim'] , _output_dim = self.parameter_dict['output_dim'],_debug=self.debug).to(self.device)
        
        self.dense = com.DenseClf(inputdim= self.parameter_dict['output_dim'],        # *2 because we have two things concatinated here
                                  hiddendim= self.parameter_dict['output_dim']/2,
                                  outputdim=self.number_of_labels).to(self.device)
        
        
    def train(self, data, optimizer, loss_fn, device):
    #
        if self.pointwise:
            return self._train_pointwise_(data, optimizer, loss_fn, device)
        else:
            return self._train_pairwise_(data, optimizer, loss_fn, device)

    def _train_pairwise_(self, data, optimizer, loss_fn, device):
        '''
            Given data, passes it through model, inited in constructor, returns loss and updates the weight
            :params data: {batch of question, pos paths, neg paths and dummy y labels}
            :params optimizer: torch.optim object
            :params loss fn: torch.nn loss object
            :params device: torch.device object

            returns loss
        '''

        # Unpacking the data and model from args
        sent_batch, y_label = data['sent_batch'], data['y_label']

        optimizer.zero_grad()

        # Encoding all the data
#         hidden = self.encoder.init_hidden(sent_batch.shape[0],self.device)
        sent_batch_encoded = self.encoder(sent_batch)
        if True:
            print(sent_batch_encoded.shape)
        # Calculating dot score
        pred = self.dense(sent_batch_encoded)


        '''
            If `y == 1` then it assumed the first input should be ranked higher
            (have a larger value) than the second input, and vice-versa for `y == -1`
        '''
        try:
#             print(torch.max(pred), torch.max(y_label))
            loss = loss_fn(pred, y_label.float())
        except RuntimeError:
            traceback.print_exc()
            print(torch.max(pred.to(cpu)), torch.max(y_label.to(cpu)))
#             print(pos_scores.shape, neg_scores.shape, y_label.shape,  ques_batch.shape, pos_batch.shape, neg_batch.shape)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), .5)
        optimizer.step()
        return loss



    def predict(self, sent, device):
        """
            Same code works for both pairwise or pointwise
        """
        with torch.no_grad():

            self.encoder.eval()
            self.dense.eval()
            
#             hidden = self.encoder.init_hidden(sent.shape[0], self.device)
            sent_encoded = self.encoder(sent.long())
            pred = self.dense(sent_encoded)


            self.encoder.train()
            self.dense.train()
            return pred

In [0]:
!wget 'http://nlp.stanford.edu/data/glove.42B.300d.zip'

--2019-02-27 00:55:42--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2019-02-27 00:55:42--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip.2’

glove.42B.300d.zip. 100%[===================>]   1.75G  4.32MB/s    in 13m 56s 

2019-02-27 01:09:39 (2.14 MB/s) - ‘glove.42B.300d.zip.2’ saved [1877800501/1877800501]



In [0]:
!unzip glove.42B.300d.zip.2

Archive:  glove.42B.300d.zip.2
replace glove.42B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: glove.42B.300d.txt      


In [23]:
vecs = np.random.randn(len(word_index),300)
embeddings_index = {}
f = open('glove.42B.300d.txt', encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 0 word vectors.


In [0]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
vecs = embedding_matrix

In [0]:
parameter_dict = {
    'number_of_layer' : 1,
    'bidirectional' : False,
    'embedding_dim' : 300,
    'max_length' : MAX_SEQUENCE_LENGTH,
    'hidden_size':256,
    'vocab_size':len(word_index),
    'dropout':0.5,
    'vectors':vecs,
    'number_of_labels' : len(all_class),
    'output_dim' : split_index
}

In [0]:
loss_fn = nn.BCELoss()
modeler = CNNDot( _parameter_dict = parameter_dict,_word_to_id=None,
                                     _device=device,_pointwise=False, _debug=False)

optimizer = optim.Adam(list(filter(lambda p: p.requires_grad, modeler.encoder.parameters()))+
                       list(filter(lambda p: p.requires_grad, modeler.dense.parameters())))

In [0]:
class SimplestSampler:
    """
        Given X and Y matrices (or lists of lists),
            it returns a batch worth of stuff upon __next__
    :return:
    """

    def __init__(self, data, bs: int = 64):
        
        try:
            assert len(data["x"]) == len(data["y"])
        except AssertionError:
            
            raise MismatchedDataError("Length of x is {len(data['x'])} while of y is {len(data['y'])}")

        self.x = data["x"]
        self.y = data["y"]
        self.n = len(self.x)
        self.bs = bs  # Batch Size

    def __len__(self):
        return self.n // self.bs - (1 if self.n % self.bs else 0)

    def __iter__(self):
        self.i, self.iter = 0, 0
        return self

    def __next__(self):
        if self.i + self.bs >= self.n:
            raise StopIteration

        _x, _y = self.x[self.i:self.i + self.bs], self.y[self.i:self.i + self.bs]
        self.i += self.bs

        return _x, _y

In [0]:
train_data = {
    'x' : train_x,
    'y': np.asarray(train_y)
}

valid_data = {
    'x' : val_x,
    'y' : np.asarray(val_y)
}

train_fact_iter = SimplestSampler(train_data)
valid_fact_iter = SimplestSampler(valid_data)

In [30]:
for dat in train_fact_iter:
    break
dat[0].shape, dat[1].shape

((64, 200), (64, 48))

In [31]:
_x = torch.tensor(dat[0], dtype=torch.long, device=device)
_y = torch.tensor(dat[1], dtype=torch.long, device=device)

data = {
    'sent_batch' : _x,
    'y_label' : _y
}

loss = modeler.train(data, optimizer, loss_fn, device)
op = modeler.predict(_x,device)

torch.Size([64, 619])


In [32]:

len(op)

64

In [33]:
loss

tensor(0.6973, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

In [0]:
def eval(op,_y):
    return torch.mean(op.round()[_y==1])

In [0]:
def get_predictions_given_text(abstract):
    global modeler, all_class_to_id, word_index
    
    text_id = np.asarray([word_index.get(word, 0) for word in abstract.split()])
    _x = torch.tensor(text_id, dtype=torch.long, device=device).unsqueeze(0)
    y_pred = modeler.predict(_x, device)
    classes = [all_class[i] for i,x in enumerate(y_pred.round().int().squeeze()) if x.item() == 1]
    terms = get_list_for_terms(classes)
    return terms

In [0]:
get_predictions_given_text(texts[1])

In [0]:
def simplest_loop(epochs: int,
                  train_data_iter,
                  valid_data_iter,
                  device: torch.device,
                  model,
                  optimizer,
                  loss_fn,
                  eval_fn= None) -> (list, list, list):
    """
        A fn which can be used to train a language model.
        The model doesn't need to be an nn.Module,
            but have an eval (optional), a train and a predict function.
        Data should be a dict like so:
            {"train":{"x":np.arr, "y":np.arr}, "val":{"x":np.arr, "y":np.arr} }
        Train_fn must return both loss and y_pred
        :param epochs: number of epochs to train for
        :param data: a dict having keys train_x, test_x, train_y, test_y
        :param device: torch device to create new tensor from data
        :param opt: optimizer
        :param loss_fn: loss function
        :param train_fn: function to call with x and y
        :param predict_fn: function to call with x (test)
        :param data_fn: a class to which we can pass X and Y, and get an iterator.
        :param eval_fn: (optional) function which when given pred and true, returns acc
        :return: traces
    """

    train_loss = []
    train_acc = []
    val_acc = []
    lrs = []

    # Epoch level
    for e in range(epochs):

        per_epoch_loss = []
        per_epoch_tr_acc = []

        # Train
        with Timer() as timer:

            # Make data

            for x, y in tqdm(train_data_iter):
                optimizer.zero_grad()

                _x = torch.tensor(x, dtype=torch.long, device=device)
                _y = torch.tensor(y, dtype=torch.long, device=device)
                
                data = {
                    'sent_batch' : _x,
                    'y_label' : _y
                }
                
                loss = model.train(data, optimizer, loss_fn, device)
                y_pred = model.predict(_x,device)

                per_epoch_tr_acc.append(eval_fn(y_pred,_y).item())
                per_epoch_loss.append(loss.item())


        # Val
        with torch.no_grad():

            per_epoch_vl_acc = []
            for x, y in tqdm(valid_data_iter):
                _x = torch.tensor(x, dtype=torch.long, device=device)
                _y = torch.tensor(y, dtype=torch.long, device=device)

                y_pred = model.predict(_x,device)

                per_epoch_vl_acc.append(eval_fn(y_pred, _y).item())

        # Bookkeep
        train_acc.append(np.mean(per_epoch_tr_acc))
        train_loss.append(np.mean(per_epoch_loss))
        val_acc.append(np.mean(per_epoch_vl_acc))

        print("Epoch: %(epo)03d | Loss: %(loss).5f | Tr_c: %(tracc)0.5f | Vl_c: %(vlacc)0.5f | Time: %(time).3f min"
              % {'epo': e,
                 'loss': float(np.mean(per_epoch_loss)),
                 'tracc': float(np.mean(per_epoch_tr_acc)),
                 'vlacc': float(np.mean(per_epoch_vl_acc)),
                 'time': timer.interval / 60.0})

    return train_acc, train_loss, val_acc

In [37]:
traces = simplest_loop(epochs= 60,
                  train_data_iter=train_fact_iter,
                  valid_data_iter=valid_fact_iter,
                  device=device,
                  model = modeler,
                  optimizer=optimizer,
                  loss_fn=loss_fn,
             eval_fn=eval)

 25%|██▌       | 2/8 [00:00<00:00, 14.35it/s]

torch.Size([64, 619])
torch.Size([64, 619])
torch.Size([64, 619])


 75%|███████▌  | 6/8 [00:00<00:00, 15.18it/s]

torch.Size([64, 619])
torch.Size([64, 619])
torch.Size([64, 619])
torch.Size([64, 619])


9it [00:00, 16.05it/s]                       
2it [00:00, 93.50it/s]               
  0%|          | 0/8 [00:00<?, ?it/s]

...

torch.Size([64, 619])
torch.Size([64, 619])
Epoch: 058 | Loss: 0.02341 | Tr_c: 0.94392 | Vl_c: 0.63245 | Time: 0.008 min
torch.Size([64, 619])
torch.Size([64, 619])


 50%|█████     | 4/8 [00:00<00:00, 19.26it/s]

torch.Size([64, 619])
torch.Size([64, 619])
torch.Size([64, 619])
torch.Size([64, 619])


9it [00:00, 19.28it/s]                       
2it [00:00, 100.59it/s]              

torch.Size([64, 619])
torch.Size([64, 619])
torch.Size([64, 619])
Epoch: 059 | Loss: 0.02371 | Tr_c: 0.94644 | Vl_c: 0.63120 | Time: 0.008 min


In [0]:
val_data =  tokenizer.sequences_to_texts(val_x)

In [0]:
predictions = [get_predictions_given_text(data_points) for index, data_points in enumerate(val_data)]

In [0]:
for dat_val in valid_fact_iter:
    break
dat_val[0].shape, dat_val[1].shape

((64, 200), (64, 48))

In [0]:
y_actual = []
for result in val_y:
  idx = np.where(result==1)
  y_actual.append(np.take((all_class), idx))

In [0]:
val_y

In [0]:
label_names = []
for result in label_vector:
  idy = np.where(result==1)
  label_names.append(np.take(all_class, idy))

In [0]:
all_labels = []
for i in label_names:
  for j in i:
    #print(j[0])
    all_labels.append(get_list_for_terms(j))

In [0]:
len(all_labels)

774

In [0]:
fulldict = dict.fromkeys(newdict, 0)
for x in range(len(all_labels)):
  for i in all_labels[x]:
      fulldict[i]=fulldict[i]+1
print(fulldict)

{'Environment and Public Health': 25, 'Nonsyndromic sensorineural hearing loss': 1, 'Immune System Phenomena': 5, 'Endocrine System Diseases': 128, 'Immune System Diseases': 87, 'Cells': 1, 'Therapeutics': 3, 'Population Characteristics': 25, 'Occupational Diseases': 1, 'Eye Diseases': 83, 'Skin and Connective Tissue Diseases': 167, 'Otorhinolaryngologic Diseases': 14, 'Musculoskeletal and Neural Physiological Phenomena': 5, 'Nutritional and Metabolic Diseases': 288, 'Mental Disorders': 118, 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities': 639, 'Health Occupations': 25, 'Nervous System Diseases': 375, 'Health Care Quality, Access, and Evaluation': 24, 'Diagnosis': 31, 'Chemically-Induced Disorders': 5, 'Reproductive and Urinary Physiological Phenomena': 3, 'Digestive System Diseases': 72, 'Male Urogenital Diseases': 125, 'Behavior and Behavior Mechanisms': 87, 'Physical Phenomena': 1, 'Musculoskeletal Diseases': 132, 'Cardiovascular Diseases': 77, 'Psychological Pheno

In [0]:
from collections import Counter

fulldict = Counter(fulldict)
fulldict.most_common()

In [0]:
newdict = Counter(newdict)
newdict.most_common()

[('Congenital, Hereditary, and Neonatal Diseases and Abnormalities', 126),
 ('Nervous System Diseases', 56),
 ('Neoplasms', 52),
 ('Nutritional and Metabolic Diseases', 36),
 ('Skin and Connective Tissue Diseases', 29),
 ('Female Urogenital Diseases and Pregnancy Complications', 25),
 ('Male Urogenital Diseases', 24),
 ('Pathological Conditions, Signs and Symptoms', 24),
 ('Endocrine System Diseases', 20),
 ('Musculoskeletal Diseases', 17),
 ('Cardiovascular Diseases', 12),
 ('Immune System Diseases', 11),
 ('Eye Diseases', 11),
 ('Digestive System Diseases', 9),
 ('Hemic and Lymphatic Diseases', 8),
 ('Mental Disorders', 7),
 ('Behavior and Behavior Mechanisms', 4),
 ('Environment and Public Health', 2),
 ('Population Characteristics', 2),
 ('Health Occupations', 2),
 ('Health Care Quality, Access, and Evaluation', 2),
 ('Diagnosis', 2),
 ('Investigative Techniques', 2),
 ('Physiological Phenomena', 2),
 ('Natural Science Disciplines', 2),
 ('Bacterial Infections and Mycoses', 1),
 ('

In [0]:
real1 = Counter(real1)
real1.most_common()

[('Congenital, Hereditary, and Neonatal Diseases and Abnormalities', 126),
 ('Nervous System Diseases', 71),
 ('Neoplasms', 55),
 ('Nutritional and Metabolic Diseases', 48),
 ('Skin and Connective Tissue Diseases', 44),
 ('Pathological Conditions, Signs and Symptoms', 43),
 ('Female Urogenital Diseases and Pregnancy Complications', 35),
 ('Male Urogenital Diseases', 34),
 ('Endocrine System Diseases', 29),
 ('Cardiovascular Diseases', 23),
 ('Musculoskeletal Diseases', 22),
 ('Immune System Diseases', 21),
 ('Eye Diseases', 17),
 ('Digestive System Diseases', 17),
 ('Mental Disorders', 13),
 ('Hemic and Lymphatic Diseases', 12),
 ('Bacterial Infections and Mycoses', 7),
 ('Behavior and Behavior Mechanisms', 6),
 ('Physiological Phenomena', 6),
 ('Environment and Public Health', 4),
 ('Population Characteristics', 4),
 ('Health Occupations', 4),
 ('Health Care Quality, Access, and Evaluation', 4),
 ('Diagnosis', 4),
 ('Investigative Techniques', 4),
 ('Genetic Phenomena', 4),
 ('Natural

In [0]:
preds1 = Counter(preds1)
preds1.most_common()

[('Congenital, Hereditary, and Neonatal Diseases and Abnormalities', 144),
 ('Nervous System Diseases', 63),
 ('Neoplasms', 54),
 ('Nutritional and Metabolic Diseases', 42),
 ('Skin and Connective Tissue Diseases', 38),
 ('Pathological Conditions, Signs and Symptoms', 32),
 ('Male Urogenital Diseases', 29),
 ('Female Urogenital Diseases and Pregnancy Complications', 29),
 ('Endocrine System Diseases', 23),
 ('Musculoskeletal Diseases', 20),
 ('Cardiovascular Diseases', 14),
 ('Immune System Diseases', 12),
 ('Eye Diseases', 11),
 ('Mental Disorders', 10),
 ('Digestive System Diseases', 10),
 ('Hemic and Lymphatic Diseases', 10),
 ('Behavior and Behavior Mechanisms', 6),
 ('Environment and Public Health', 2),
 ('Population Characteristics', 2),
 ('Health Occupations', 2),
 ('Health Care Quality, Access, and Evaluation', 2),
 ('Diagnosis', 2),
 ('Investigative Techniques', 2),
 ('Physiological Phenomena', 2),
 ('Natural Science Disciplines', 2),
 ('Bacterial Infections and Mycoses', 1),


In [0]:
newdict = {k: v for v, k in enumerate(targets)}

In [0]:
newdict

{'Bacterial Infections and Mycoses': 42,
 'Behavior and Behavior Mechanisms': 24,
 'Behavioral Disciplines and Activities': 32,
 'Biological Phenomena': 37,
 'Cardiovascular Diseases': 27,
 'Cell Physiological Phenomena': 34,
 'Cells': 5,
 'Chemically-Induced Disorders': 20,
 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities': 15,
 'Diagnosis': 19,
 'Digestive System Diseases': 22,
 'Endocrine System Diseases': 3,
 'Environment and Public Health': 0,
 'Eye Diseases': 9,
 'Female Urogenital Diseases and Pregnancy Complications': 33,
 'Fluids and Secretions': 47,
 'Genetic Phenomena': 43,
 'Health Care Quality, Access, and Evaluation': 18,
 'Health Occupations': 16,
 'Hemic and Lymphatic Diseases': 38,
 'Immune System Diseases': 4,
 'Immune System Phenomena': 2,
 'Investigative Techniques': 41,
 'Male Urogenital Diseases': 23,
 'Mental Disorders': 14,
 'Musculoskeletal Diseases': 26,
 'Musculoskeletal and Neural Physiological Phenomena': 12,
 'Natural Science Disciplines':

In [0]:
actual_labels

In [0]:
actual_labels = []
for i in y_actual:
  for j in i:
    #print(j[0])
    actual_labels.append(get_list_for_terms(j))

In [0]:
predictions

In [0]:
newdict = dict.fromkeys(newdict, 0)
for x in range(len(predictions)):
  for i in predictions[x]:
    if i in actual_labels[x]:
      newdict[i]=newdict[i]+1
print(newdict['Cardiovascular Diseases'])

12


In [0]:
preds1 = dict.fromkeys(newdict, 0)
for x in range(len(predictions)):
  for i in predictions[x]:
      preds1[i]=preds1[i]+1
print(preds1)


{'Environment and Public Health': 2, 'Nonsyndromic sensorineural hearing loss': 0, 'Immune System Phenomena': 0, 'Endocrine System Diseases': 23, 'Immune System Diseases': 12, 'Cells': 0, 'Therapeutics': 0, 'Population Characteristics': 2, 'Occupational Diseases': 0, 'Eye Diseases': 11, 'Skin and Connective Tissue Diseases': 38, 'Otorhinolaryngologic Diseases': 0, 'Musculoskeletal and Neural Physiological Phenomena': 0, 'Nutritional and Metabolic Diseases': 42, 'Mental Disorders': 10, 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities': 144, 'Health Occupations': 2, 'Nervous System Diseases': 63, 'Health Care Quality, Access, and Evaluation': 2, 'Diagnosis': 2, 'Chemically-Induced Disorders': 0, 'Reproductive and Urinary Physiological Phenomena': 0, 'Digestive System Diseases': 10, 'Male Urogenital Diseases': 29, 'Behavior and Behavior Mechanisms': 6, 'Physical Phenomena': 0, 'Musculoskeletal Diseases': 20, 'Cardiovascular Diseases': 14, 'Psychological Phenomena': 0, 'Res

In [0]:
newdict

{'Bacterial Infections and Mycoses': 1,
 'Behavior and Behavior Mechanisms': 4,
 'Behavioral Disciplines and Activities': 0,
 'Biological Phenomena': 0,
 'Cardiovascular Diseases': 12,
 'Cell Physiological Phenomena': 0,
 'Cells': 0,
 'Chemically-Induced Disorders': 0,
 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities': 126,
 'Diagnosis': 2,
 'Digestive System Diseases': 9,
 'Endocrine System Diseases': 20,
 'Environment and Public Health': 2,
 'Eye Diseases': 11,
 'Female Urogenital Diseases and Pregnancy Complications': 25,
 'Fluids and Secretions': 0,
 'Genetic Phenomena': 0,
 'Health Care Quality, Access, and Evaluation': 2,
 'Health Occupations': 2,
 'Hemic and Lymphatic Diseases': 8,
 'Immune System Diseases': 11,
 'Immune System Phenomena': 0,
 'Investigative Techniques': 2,
 'Male Urogenital Diseases': 24,
 'Mental Disorders': 7,
 'Musculoskeletal Diseases': 17,
 'Musculoskeletal and Neural Physiological Phenomena': 0,
 'Natural Science Disciplines': 2,
 'Neopla

In [0]:
real1 = dict.fromkeys(newdict, 0)
for x in range(len(actual_labels)):
  for i in actual_labels[x]:
      real1[i]=real1[i]+1
print(real1)

{'Environment and Public Health': 4, 'Nonsyndromic sensorineural hearing loss': 0, 'Immune System Phenomena': 0, 'Endocrine System Diseases': 29, 'Immune System Diseases': 21, 'Cells': 0, 'Therapeutics': 0, 'Population Characteristics': 4, 'Occupational Diseases': 0, 'Eye Diseases': 17, 'Skin and Connective Tissue Diseases': 44, 'Otorhinolaryngologic Diseases': 2, 'Musculoskeletal and Neural Physiological Phenomena': 0, 'Nutritional and Metabolic Diseases': 48, 'Mental Disorders': 13, 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities': 126, 'Health Occupations': 4, 'Nervous System Diseases': 71, 'Health Care Quality, Access, and Evaluation': 4, 'Diagnosis': 4, 'Chemically-Induced Disorders': 1, 'Reproductive and Urinary Physiological Phenomena': 0, 'Digestive System Diseases': 17, 'Male Urogenital Diseases': 34, 'Behavior and Behavior Mechanisms': 6, 'Physical Phenomena': 0, 'Musculoskeletal Diseases': 22, 'Cardiovascular Diseases': 23, 'Psychological Phenomena': 0, 'Res

In [0]:
string1 = val_data
string2 = predictions 
string3 = actual_labels

with open("results_cnn_superclass.csv","w") as f:
  writer = csv.writer(f)
  writer.writerows(zip(string1,string2,string3))

In [0]:
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\stuti.nayak\AppData\Local\Continuum\Anaconda3\envs\metawiz-keras-theano\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [0]:
df = pandas.read_csv("results_cnn_disease_names.csv")
df.columns = ["abstract", "preds", "actual_labels"]


In [0]:
predict=df["preds"].tolist()
real=df["actual_labels"].tolist()


In [0]:
newdict2 = {k: v for v, k in enumerate(targets)}

In [0]:
#Now chanign the hiearchy of the diseases dataset where we only consider Diseases ontology 
clean_data_ds = []
us = []
for c in tqdm(clean_data):
    try:
        labels = []
        for l in c[1]:
            #print(l)
            if 'D' == l[0]:
                labels = labels + l.replace('+','|').split("|")
                #labels = labels + l.split('+')
         #change the class hiearchy 
        #new_labels = list(set(get_classes_for_dois(labels)))
        clean_data_ds.append((c[0], list(set(labels))))
    except:
        us.append(c)

clean_data1 = clean_data_ds

100%|██████████| 793/793 [00:00<00:00, 79216.97it/s]


In [0]:
all_disease = list(set([g for f in clean_data1 for g in f[1]]))

In [0]:
newdict2 = {k: v for v, k in enumerate(all_disease)}
newdict2 = dict.fromkeys(newdict2, 0)
for x in range(len(predict)):
  for i in predict[x]:
    print(newdict2[i])
#     if i in real[x]:
#       print(newdict2[i])
#       newdict2[i]=newdict2[i]+1

# print(newdict2)


KeyError: ignored

In [0]:
clean_data1[0]

IndexError: ignored